# Jorual DAO: A New Framework for DeSci in Web 3.0

In [1]:
from math import floor
import pandas as pd
from random import randint

## type

In [16]:
class IUser():
  address: int
  token: int
  def __init__(self, address:str, token:int) -> None:
    self.address = address
    self.token = token

class IReview(IUser):
  rating: int
  def __init__(self, address:str, rating:int) -> None:
    self.rating = rating
    super().__init__(address, 0)

class IAuthor(IUser):
  sequence: int
  def __init__(self, address:str, sequence:int) -> None:
    self.sequence = sequence
    super().__init__(address, 0)

class ICitedBy(IUser):
  articleID: int
  def __init__(self, address:str, articleID:int) -> None:
    self.articleID = articleID
    super().__init__(address, 0)



## dao class

In [46]:
class ArticleDAO:
  _distribution:list[IUser]
  _articleID = None
  _authors:list[IAuthor]
  _reviewers:list[IReview]
  _downloads:list[IUser]
  _cites:list[ICitedBy]
  _log = pd.DataFrame()

  def __init__(self, articleID:int, authors:list[str]):
    self._articleID = articleID
    self._authors = list()
    for i, author in enumerate(authors):
      self._authors.append(IAuthor(author, i))
    self._reviewers = list()
    self._downloads = list()
    self._cites = list()
    self._distribution = list()
  def review(self, reviews:list[IReview]):
    token = 0
    for review in reviews:
      rating = 0
      if review.rating >= 5:
        rating = 5
      elif review.rating <= 1:
        rating = 1
      token += rating
      review.token = (6 - rating) * 1000
    self._reviewers = reviews
    author_len = len(self._authors)

    token = token * 1000
    weight = dict()
    weight_len = 0
    for i, author in enumerate(self._authors):
      n = author_len - i
      # if i == 1:
      #   n = author_len
      weight[i] = n
      weight_len += n
    for i, author in enumerate(self._authors):
      author.token = floor(token * (weight[i] / weight_len))
      # self._authors.append(IAuthor(author.address, floor(token * (weight[i] / weight_len))))
  def download(self, address:str):
    user = next((obj for obj in self._downloads if obj.address == address), None)
    if user:
      return
      # user.token = user.token + 1
    else:
      self._downloads.append(IUser(address, 1))
    for author in self._authors:
      tmp = next((obj for obj in self._authors if obj.address == author.address), None)
      tmp.token = tmp.token + 1
    self.record()
  def cite(self, address:str):
    user = next((obj for obj in self._cites if obj.address == address), None)
    if user:
      user.token = user.token + 10
    else:
      self._cites.append(IUser(address, 10))
    for author in self._authors:
      tmp = next((obj for obj in self._authors if obj.address == author.address), None)
      tmp.token = tmp.token + 2
    self.record()
  def record(self):
    data = {}
    total = 0
    for obj in self._authors:
      data[obj.address] = [obj.token]
      total += obj.token

    for obj in self._reviewers:
      data[obj.address] = [obj.token]
      total += obj.token

    for obj in self._cites:
      data[obj.address] = [obj.token]
      total += obj.token

    if len(self._downloads) > 0:
      data['downloader'] = [self._downloads[0].token]
    # data['download_total'] = len(self._downloads)
    
    total += len(self._downloads) * 1
    row = pd.DataFrame(data)
    # self._log = pd.concat([self._log, row], ignore_index=True)
    frac = row.divide(total, axis=0)
    self._log = pd.concat([self._log, frac], ignore_index=True)
  def emulate(self, download:int, cite:int):
    arr = []
    for i in range(download):
      arr.append({'address':'u' + str(i), 'type':'d'})
    for i in range(cite):
      index = randint(0, len(arr) - 1)
      arr.insert(index, {'address':'u' + str(randint(0,index)), 'type':'c'})

    for d in arr:
      if d['type'] == 'd':
        self.download(d['address'])
      else:
        self.cite(d['address'])
  def show(self):
    return self._log

## run instance

In [48]:
article = ArticleDAO(1, ['a1','a2','a3'])
article.review([IReview(address='r1', rating=5), IReview(address='r2', rating=4)])
article.emulate(100, 10)

article.show()

,a1,a2,a3,r1,r2,downloader,u3,u5,u14,u22,u17,u12,u37,u38,u1,u86
0,0.208365,0.138882,0.069483,0.083313,0.499875,0.000083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.208378,0.138919,0.069543,0.083285,0.499709,0.000083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.208392,0.138956,0.069603,0.083257,0.499542,0.000083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.208281,0.138937,0.069677,0.083146,0.498878,0.000083,0.000831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.208295,0.138974,0.069737,0.083119,0.498712,0.000083,0.000831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0.208563,0.142071,0.075660,0.079726,0.478354,0.000080,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797
106,0.208576,0.142106,0.075715,0.079700,0.478202,0.000080,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797
107,0.208589,0.142140,0.075771,0.079675,0.478050,0.000080,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797,0.000797
108,0.208602,0.142174,0.075826,0.079650,0.477897,0.000080,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796


In [11]:
df = pd.DataFrame({'a':[1], 'b':[2]})
# frac = df.divide(df.sum(axis=1), axis=0)
df.sum(axis=1).values[0]

3